In [1]:
import numpy as np
from scipy import misc
import pprint as pp
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf
from keras.datasets import mnist
from keras.models import Sequential
from keras.models import Model
from keras.models import load_model
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.applications.resnet50 import ResNet50
from keras.optimizers import Adam
from keras import backend as Keras

Using TensorFlow backend.


In [2]:
import sys
sys.path.insert(0, '/home/albert/github/DenseNet/')
import densenet

### Mine Triplets

In [3]:
CAFFE_ROOT = '/home/albert/caffe/'
img_dir = os.listdir(CAFFE_ROOT + 'data/market-1501/bounding_box_train')

In [4]:
train_files = {}
train_arr = []
labels = []

for f in img_dir:
    if f[-4:] == '.jpg':
        idt = int(f[0:f.index('_')])
        if not any(idt == l for l in labels):
            labels.append(idt)
            train_files[idt] = []
        path = CAFFE_ROOT + 'data/market-1501/bounding_box_train/' + f
        train_files[idt].append(path)
        train_arr.append([path, idt])

labels.sort()

In [5]:
def batch_generator(train_files, labels, P=18, K=4):
    while True:
        batch = []
        idt_choice = np.random.choice(labels, P, replace=False)
        for p in range(len(idt_choice)):
#             batch.append([])
            k_choice = np.random.choice(range(len(train_files[idt_choice[p]])), K, replace=True)
            for k in k_choice:
                path = train_files[idt_choice[p]][k]
                img = cv2.resize(misc.imread(path), (224, 224))
                batch.append(img.tolist())
        yield(batch)

In [6]:
# generator = batch_generator(train_files, labels)
# batch = generator.next()
# batch = np.array(batch, dtype=np.uint8)

# plt.figure(figsize=(8,10))
# t = 0
# for idt in range(20):
#     t += 1
#     plt.subplot(5, 4, t)
#     plt.imshow(batch[idt])
# plt.show()

### Train Network

In [7]:
# Number of identities
P_param = 5
# Number of images per identity
K_param = 4

In [8]:
def preprocess(img):
    VGG_MEAN = [103.939, 116.779, 123.68]
    out = np.copy(img) * 255
    out = out[:, :, [2,1,0]] # swap channel from RGB to BGR
    out[:,:,0] -= VGG_MEAN[0]
    out[:,:,1] -= VGG_MEAN[1]
    out[:,:,2] -= VGG_MEAN[2]
    return out

In [9]:
def output_batch_generator(train_files, labels, P=P_param, K=K_param):
    while True:
        batch = []
        idt_choice = np.random.choice(labels, P, replace=False)
        for p in range(len(idt_choice)):
            # n_choose = np.minimum(K, len(train_files[idt_choice[p]]))
            k_choice = np.random.choice(range(len(train_files[idt_choice[p]])), K, replace=True)
            for k in k_choice:
                path = train_files[idt_choice[p]][k]
                img = cv2.resize(misc.imread(path), (224, 224)).astype(np.float64)
#                 img = preprocess(img)
                batch.append(img.tolist())
        output = np.array(batch)
        yield(output, np.zeros((P*K, 128)))

In [10]:
def log1p(x):
    return Keras.log(1 + Keras.exp(x))

In [11]:
def dist(x1, x2):
    return Keras.sum(Keras.abs(x1 - x2))

In [12]:
def triplet_loss(y_true, y_pred, margin=0.1, P=P_param, K=K_param, output_dim = 128):
    embeddings = Keras.reshape(y_pred, (-1, output_dim))

    loss = tf.Variable(1, dtype=tf.float32)

    for i in range(P):
        for a in range(K):
            pred_anchor = embeddings[i*K + a]
            hard_pos = Keras.max(dist(pred_anchor, embeddings[i*K:(i + 1)*K]))
            hard_neg = Keras.min(dist(pred_anchor, Keras.concatenate([embeddings[0:i*K],
                                                                      embeddings[(i + 1)*K:]], 0)))
            loss += log1p(hard_pos - hard_neg)
    return loss

In [13]:
def evaluate_rank(net, rank, all_embeddings, all_identities, test_iter=1000):
    correct = 0
    f_choice = np.random.choice(range(len(train_arr)), np.minimum(test_iter, len(train_arr)), replace=False)
    for f in f_choice:
        query_img = cv2.resize(misc.imread(train_arr[f][0]), (224,224))
        query_embedding = net.predict(query_img.reshape(1, 224, 224, 3))
        distance_vectors = np.squeeze(np.abs(all_embeddings - query_embedding))
        distance = np.sum(distance_vectors, axis=1)
        top_inds = distance.argsort()[:rank+1]
        output_classes = np.array(all_identities)[top_inds].astype(np.uint16)
        
#         pp.pprint(zip(distance[top_inds], np.array(all_identities)[top_inds].astype(np.uint16)))
        
        i = 0
        for c in output_classes:
            if c == int(train_arr[f][1]):
                i += 1
        if i > 1:
            correct += 1
#         print(correct)
    return float(correct)/test_iter

According to the triplet loss paper, use an adaptive learning rate decay that is constant at first, then decays exponentially.

In [14]:
# https://medium.com/towards-data-science/learning-rate-schedules-and-adaptive-learning-rate-methods-for-deep-learning-2c8f433990d1
from keras.callbacks import LearningRateScheduler

def exp_decay(epoch):
    initial_lr = 0.0003
    t_0 = 0
#     t_1 = 25000
    if epoch <= t_0:
        return initial_lr
    elif epoch > t_0:
        return initial_lr * 0.001 ** (epoch/100)

lrate = LearningRateScheduler(exp_decay)

In [15]:
score_arr = []

Replace top layer of ResNet with a FC layer (1024) with batch normalization and ReLU and a FC layer (128). Train with all layers as learnable for 50 epochs with learning rate decay 1e-6.

In [16]:
# https://keras.io/applications/#fine-tune-inceptionv3-on-a-new-set-of-classes
i = 0
# for epochs in range(10, 20, 10):
image_dim = (224, 224, 3)
base_model = densenet.DenseNetImageNet121(image_dim)
print(len(base_model.layers))

Weights for the model were loaded successfully
427


In [17]:
base_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 112, 112, 64)  9408        input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 112, 112, 64)  256         conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 112, 112, 64)  0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [18]:
# base_model.compile(loss=triplet_loss, optimizer=Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6))

In [19]:
base_model.outputs = [base_model.layers[-2].output]
base_model.layers[-1].outbound_nodes = []

In [20]:
print(base_model.outputs)

[<tf.Tensor 'global_average_pooling2d_1/Mean:0' shape=(?, 1024) dtype=float32>]


In [21]:
x = base_model.outputs[0]
x = Dense(1024)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
predictions = Dense(128)(x)

trinet = Model(inputs=base_model.input, outputs=predictions)

trinet.compile(loss=triplet_loss, optimizer=Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6))

In [22]:
for layer in trinet.layers:
    layer.trainable = True

In [23]:
trinet.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 112, 112, 64)  9408        input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 112, 112, 64)  256         conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 112, 112, 64)  0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [24]:
epochs = 50
trinet.fit_generator(output_batch_generator(train_files, labels), steps_per_epoch=5, epochs=epochs)

Epoch 1/50


ResourceExhaustedError: OOM when allocating tensor with shape[20,14,14,128]
	 [[Node: conv2d_80/convolution = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/gpu:0"](activation_80/Relu, conv2d_80/kernel/read)]]
	 [[Node: loss/add_61/_5155 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_75789_loss/add_61", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'conv2d_80/convolution', defined at:
  File "/home/albert/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/albert/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/albert/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py", line 16, in <modu

Load model if needed. Continue training net, but now with adaptive learning rate.

In [20]:
# # https://github.com/fchollet/keras/issues/3977
# # Load model with custom loss (object)
# trinet = load_model('/home/albert/github/tensorflow/trinet_learn_all_epochs_50.h5',
#                    custom_objects={'triplet_loss':triplet_loss})

In [21]:
# trinet.summary()

In [22]:
trinet.compile(loss=triplet_loss, optimizer=Adam(lr=0.0003, beta_1=0.5, beta_2=0.999, epsilon=1e-08, decay=0.0))

In [23]:
epochs = 50
trinet.fit_generator(output_batch_generator(train_files, labels), 
                     steps_per_epoch=5, 
                     epochs=epochs, callbacks=[lrate])

Epoch 1/50
5/5 [==============================] - 5s - loss: 1.0000     
Epoch 2/50
5/5 [==============================] - 4s - loss: 1.0000     
Epoch 3/50
5/5 [==============================] - 4s - loss: 1.0000     
Epoch 4/50
5/5 [==============================] - 4s - loss: 1.0000     
Epoch 5/50
5/5 [==============================] - 4s - loss: 1.0000     
Epoch 6/50
5/5 [==============================] - 4s - loss: 1.0000     
Epoch 7/50
5/5 [==============================] - 4s - loss: 1.0000     
Epoch 8/50
5/5 [==============================] - 4s - loss: 1.0000     
Epoch 9/50
5/5 [==============================] - 4s - loss: 1.0000     
Epoch 10/50
5/5 [==============================] - 4s - loss: 1.0000     
Epoch 11/50
5/5 [==============================] - 4s - loss: 1.0000     
Epoch 12/50
5/5 [==============================] - 4s - loss: 1.0000     
Epoch 13/50
5/5 [==============================] - 4s - loss: 1.0000     
Epoch 14/50
5/5 [==============================

In [ ]:
# trinet.save('/home/albert/github/tensorflow/trinet_learn_all_epochs_adalar_%d.h5' % epochs)

In [26]:
all_embeddings = []
all_identities = []
for idt in train_files.keys():
    for f in train_files[idt]:
        img = cv2.resize(misc.imread(f), (224,224))
        predict = trinet.predict(img.reshape(1, 224, 224, 3))
        all_embeddings.append(predict)
        all_identities.append(idt)

In [27]:
i = 0
score_arr.append([])
for x in range(3):
    score = evaluate_rank(trinet, 20, all_embeddings, all_identities, test_iter=1000)
    print score
    score_arr[i].append(score)
i += 1

0.53
0.535
0.538


In [26]:
# np.save('/home/albert/github/tensorflow/rank20_learn_all.npy', np.array(score_arr))

In [27]:
print score_arr

[[0.136, 0.153, 0.148]]


https://keras.io/getting-started/faq/: use load_model to reinstantiate model.

Train all, lr_decay=1e-6, use log1p

### Evaluate Performance

In [ ]:
p = 7
query_img = cv2.resize(misc.imread(train_files[p][2]), (224,224))
query_embedding = trinet.predict(model.predict(query_img.reshape(1, 224, 224, 3)))

In [ ]:
distance_vectors = np.squeeze(np.abs(all_embeddings - query_embedding))
distance = np.sum(distance_vectors, axis=1)

In [ ]:
top_inds = distance.argsort()[:20]
output_classes = np.array(all_identities)[top_inds]
pp.pprint(zip(distance[top_inds], np.array(all_identities)[top_inds].astype(np.uint16)))